In [ ]:
import numpy as np
import Slope_Tracer


In [ ]:
import imp
imp.reload(Slope_Tracer)

In [ ]:
input_dict = dict(Slope_Tracer.default_input_dict)
input_dict['Ttot'] = 800
input_dict['AH'] = 10.
Slope_Tracer.run_sim('test/',plot=True,**input_dict)

In [ ]:
('%01d' % np.log10(1.e-3)).replace('-','m')

In [ ]:
    AHs = [10.,100.]
    ADVs = [0,0]
    Kinfs = [1.e-3,1.e-3];


In [ ]:
for ii in range(len(AHs)):

    outdir = ('z0_0p5000_AH_%03d_ADV_%01d_Kinf_%01d/' % (AHs[ii],ADVs[ii],np.log10(Kinfs[ii]))).replace('-','m')
    print(outdir)